# 01 · Markov Blankets from Samples (Approximate)

Goal: demonstrate an *approximate* Markov-blanket partition emerging from continuous dynamics by checking conditional independencies on samples.

We:
1. Simulate a 2D SDE (double-well + swirl) to a nonequilibrium steady state (NESS).
2. Construct variables $(I, A, S, E)$ from trajectory slices.
3. Compute precision-matrix block norms and the **Schur-complement** block $K_{IE\mid AS}$.

> Blanket intuition: under suitable partition/coarse-graining, internal $I$ and external $E$ become (approximately) conditionally independent given the blanket $(A,S)$.

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from persystems.sde import euler_maruyama, drift_doublewell_with_swirl
from persystems.blankets import blanket_block_norms
np.random.seed(7)

## 1) Simulate SDE to NESS
We simulate the 2D Langevin SDE using Euler–Maruyama and discard an initial burn-in window.

In [ ]:
T = 200_000
dt = 0.002
D = 0.05
f = drift_doublewell_with_swirl(a=1.0, b=1.0, k=1.0)
X = euler_maruyama(np.array([0.0, 0.5]), f, dt=dt, T=T, D=D)
burn = int(0.2*T)
Xs = X[burn:]
plt.figure(figsize=(5,4))
plt.plot(Xs[::200,0], Xs[::200,1], '.', ms=1)
plt.title('Subsampled trajectory after burn-in')
plt.xlabel('x'); plt.ylabel('y'); plt.tight_layout(); plt.show()

## 2) Build $(I,A,S,E)$ variables from the trajectory
- Here we choose: $I=x_t$, $A=\dot x_t\approx (x_{t+1}-x_t)/\Delta t$, $S=y_t$, $E=y_{t+1}$.
- This is just one **operational** partition. Other choices (e.g., coarse-grained or multi-timescale) can strengthen blanket separation.

In [ ]:
x = Xs[:-1,0]
y = Xs[:-1,1]
vx = (Xs[1:,0] - Xs[:-1,0]) / dt
y_next = Xs[1:,1]
Z = np.stack([x, vx, y, y_next], axis=1)  # columns = [I, A, S, E]
Z.shape

## 3) Blanket diagnostics: precision-matrix blocks and Schur complement
For Gaussian proxies, zeros in precision-matrix blocks correspond to conditional independencies.
We compute Frobenius norms of off-diagonal blocks and the **conditioned** $K_{IE\mid AS}$ via a Schur complement.

> Heuristic outcome we look for: $\lVert K_{IE\mid AS}\rVert$ should be **smaller** than $\lVert K_{IE}\rVert$ if the blanket partition is appropriate.

In [ ]:
norms = blanket_block_norms(Z, dims={"I":1, "A":1, "S":1, "E":1}, reg=1e-4)
pd.DataFrame([{k: float(v) for k,v in norms.items()}]).T.rename(columns={0: 'Frobenius norm'})

### Quick check
We expect (heuristically) that conditioning on the blanket $(A,S)$ reduces the residual coupling between $I$ and $E$.

In [ ]:
print('K_IE       =', norms['K_IE'])
print('K_IE|AS    =', norms['K_IE_cond_AS'])
print('Conditioning reduced IE coupling? →', norms['K_IE_cond_AS'] <= norms['K_IE'] + 1e-9)

## Notes
- Blanket structure here is **approximate** and **scale-dependent**. Coarse-graining (spatial/temporal) or different variable groupings can enhance the separation.
- In Friston’s Bayesian mechanics framing, at NESS and with a valid blanket, internal flows can be interpreted as **gradient flows** on a free-energy functional, hence the connection to *inference-like* dynamics.